# Building And Testing A Complete Trading System

In [9]:
# import yahoo
import yfinance as yf
import pandas_ta as pa
import plotly.graph_objects as go
import numpy as np

def get_data(symbol: str):
    data = yf.download(tickers=symbol, period='1000d', interval='1d')
    data.reset_index(inplace=True) 
    return data
# Get the data
#data = get_data('BTC-USD')
data = get_data('BTC-USD')

In [10]:
#Import ticker
import os
import pandas as pd

my_custom_tickers=[]
ticker ="NVDA.csv"
directory = os.path.expanduser('~/Documents/tempTestFiles2')
file_path = os.path.join(directory, ticker) 
data = pd.read_csv(file_path)


In [11]:
data

,Date,Open,High,Low,Close,Volume
0,2003-12-29,1.9308,1.9817,1.9217,1.9750,75676800.0
1,2003-12-30,1.9542,2.0167,1.9525,1.9792,55118400.0
2,2003-12-31,1.9783,1.9875,1.9017,1.9333,49736400.0
3,2004-01-02,1.9642,1.9908,1.9233,1.9233,43640400.0
4,2004-01-05,1.9525,1.9992,1.9350,1.9858,57544800.0
...,...,...,...,...,...,...
5051,2024-01-29,612.4500,624.8900,609.0700,624.6500,23019819.0
5052,2024-01-30,629.0500,634.9300,622.6000,627.7400,28714519.0
5053,2024-01-31,614.4000,622.7000,607.0000,615.2700,31427138.0
5054,2024-02-01,621.0700,631.9100,616.5000,630.2700,25952593.0


## 1- Add rejection signal

In [12]:
def identify_rejection(data):
    # Create a new column for rejection signal
    data['rejection'] = data.apply(lambda row: 2 if (
        ( (min(row['Open'], row['Close']) - row['Low']) > (1.5 * abs(row['Close'] - row['Open']))) and 
        (row['High'] - max(row['Close'], row['Open'])) < (0.8 * abs(row['Close'] - row['Open'])) and 
        (abs(row['Open'] - row['Close']) > row['Open'] * 0.001)
    ) else 1 if (
        (row['High'] - max(row['Open'], row['Close'])) > (1.5 * abs(row['Open'] - row['Close'])) and 
        (min(row['Close'], row['Open']) - row['Low']) < (0.8 * abs(row['Open'] - row['Close'])) and 
        (abs(row['Open'] - row['Close']) > row['Open'] * 0.001)
    ) else 0, axis=1)

    return data

data = identify_rejection(data)


In [13]:
data[data["rejection"]!=0]

,Date,Open,High,Low,Close,Volume,rejection
28,2004-02-09,1.9042,1.9417,1.8825,1.8833,37400400.0,1
32,2004-02-13,1.9125,1.9908,1.8917,1.9417,174774000.0,1
33,2004-02-17,1.9500,1.9692,1.9225,1.9633,57645600.0,2
40,2004-02-26,1.9025,1.9058,1.8642,1.8875,34500000.0,2
44,2004-03-03,1.8417,1.8417,1.7925,1.8225,55248000.0,2
...,...,...,...,...,...,...,...
5025,2023-12-19,494.2000,497.0000,488.9600,496.0400,3887849.0,2
5027,2023-12-21,488.0000,490.9000,484.2100,489.9000,3201984.0,2
5031,2023-12-29,498.1300,499.9700,487.5100,495.2200,28665614.0,2
5043,2024-01-17,563.4500,564.7100,547.4000,560.5300,34628976.0,2


In [14]:
def pointpos(x, xsignal):
    if x[xsignal]==1:
        return x['High']+1e-4
    elif x[xsignal]==2:
        return x['Low']-1e-4
    else:
        return np.nan

def plot_with_signal(dfpl):

    fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                    open=dfpl['Open'],
                    high=dfpl['High'],
                    low=dfpl['Low'],
                    close=dfpl['Close'])])

    fig.update_layout(
        autosize=False,
        width=1000,
        height=800, 
        paper_bgcolor='black',
        plot_bgcolor='black')
    fig.update_xaxes(gridcolor='black')
    fig.update_yaxes(gridcolor='black')
    fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                    marker=dict(size=8, color="MediumPurple"),
                    name="Signal")
    fig.show()

data['pointpos'] = data.apply(lambda row: pointpos(row,"rejection"), axis=1)
plot_with_signal(data[10:110])

## 2- Support and Resistance FUNCTIONS

In [150]:
def support(df1, l, n1, n2): #n1 n2 number candles before and after candle l
    if ( df1.Low[l-n1:l].min() < df1.Low[l] or
        df1.Low[l+1:l+n2+1].min() < df1.Low[l] ):
        return 0 # is not support
    return 1 #is support 

def resistance(df1, l, n1, n2): #n1 n2 before and after candle l
    if ( df1.High[l-n1:l].max() > df1.High[l] or
       df1.High[l+1:l+n2+1].max() > df1.High[l] ):
        return 0
    return 1

## 3- Close to resistance and support testing

### This code represents a trading strategy that identifies potential support and resistance levels in a financial market. It checks if a PARTICULAR Candlestick is close enough to a resistance level AND if the PREVIOUS CANDLES are below the resistance level.

### In simple terms, the strategy aims to find situations where the price of an asset approaches a resistance level, indicating a potential reversal or a barrier to further upward movement.

## In summary, these conditions (c1, c2, c3, and c4) collectively determine if a candlestick is close enough to a resistance level and if a part of the candlestick is still below the resistance level, indicating a potential trading opportunity according to the strategy.

### Now, let's break down the meaning of c1, c2, c3, and c4 inside the closeResistance function:

    # c1 checks if the absolute DIFFERENCE between A)the high of the candle and B) the nearest resistance level is within a specified threshold (lim). It determines if the high of the candle is close enough to the resistance level.
    # c1 = abs(df['High'][l] - min(levels, key=lambda x: abs(x - df['High'][l]))) <= lim 

        min(levels, key=lambda x: abs(x - df['High'][l]))
        is an expression that FINDS THE RESISTANCE LEVEL CLOSEST TO THE HIGH of the candle. It iterates over the levels list and calculates the absolute difference between each resistance level (x) and the high of the candle (df['High'][l]). The min() function with the key parameter is used to FIND THE MINIMUM DIFFERENCE, effectively identifying the nearest resistance level.

        abs(df['High'][l] - min(levels, key=lambda x: abs(x - df['High'][l]))) 
        calculates the absolute difference between the high of the candle and the nearest resistance level.

        <= lim 
        compares the calculated absolute difference with the specified threshold lim. If the absolute difference is less than or equal to the threshold, c1 is set to True; otherwise, it is set to False.

        THE THRESHOLD (LIM) IS USED TO DEFINE A RANGE WITHIN WHICH THE CANDLE IS CONSIDERED "CLOSE ENOUGH" TO THE RESISTANCE LEVEL. It provides flexibility in determining the proximity required for a potential trading opportunity. If the absolute difference between the high of the candle and the nearest resistance level falls within the threshold range, c1 will be True

    # c2 checks if the absolute difference between the maximum of the open and close prices of the candle (the part of the candle near the resistance level) and the nearest resistance level is within the specified threshold (lim). IT DETERMINES IF THE BODY OF THE CANDLE IS BELOW THE THRESHOLD.

    # c3 checks if the minimum of the open and close prices of the candle is less than the nearest resistance level. It verifies if a part of the candle is still below the resistance level.

    # c4 checks if the low of the candle is less than the nearest resistance level. It is similar to c3 but considers the low price instead.


In [151]:
def closeResistance(l, levels, lim, df): # l candle we consider, levels list resistence levels, lim(it) is a threeshold # ex input     cR = closeResistance(l, rrss, df.Close[l]*0.003, df) 
    if len(levels) == 0: #nothing to test
        return 0
    resistanceSpecificLevelClosestToHighofCandle = min(levels, key=lambda x: abs(x - df['High'][l]))
    isCandleBodyCloseBelowTheResistance= abs(max(df['Open'][l], df['Close'][l]) - resistanceSpecificLevelClosestToHighofCandle) <= lim 
    
    isTheHighOfTheCandleCloseEnoughToTheResistence =abs(df['High'][l] - resistanceSpecificLevelClosestToHighofCandle) <= lim
    aPartOfTheBodyCandleIsStillBelowTheResistance= min(df['Open'][l], df['Close'][l]) < resistanceSpecificLevelClosestToHighofCandle

    lowOfCandleIsStillBelowTheResistance=df['Low'][l] < resistanceSpecificLevelClosestToHighofCandle
    # the c1 and c2 are to check if the candle is near the level of resistence
    
    #absolute difference between high of the candle and nearest resistence level is in the specified lim(it)
    
    #is going to calculate the closest high  minus  to the level of the candle 
    c1 = isTheHighOfTheCandleCloseEnoughToTheResistence 
    #body candle  is below the limit threeshold

    #max is the max of open and close price(the part of the candle near resistance) and we subtract 
    c2 = isCandleBodyCloseBelowTheResistance
    # candle is close enough to resistance, we do not test a breakout of resistance, basically if part of the candle is still below resistence level
    c3 = aPartOfTheBodyCandleIsStillBelowTheResistance 
    # the same as c3 but we take the low (please notice if condition c3 is true also c4 will be true)
    c4 = lowOfCandleIsStillBelowTheResistance
    ##if candle is near resistence and a part of it is still below the level
    if (c1 or c2) and c3 and c4:
        return min(levels, key=lambda x: abs(x - df['High'][l])) # we return the level that is concerned    
    else:
        return 0

def closeSupport(l, levels, lim, df):
    if len(levels) == 0:
        return 0
    c1 = abs(df['Low'][l] - min(levels, key=lambda x: abs(x - df['Low'][l]))) <= lim
    c2 = abs(min(df['Open'][l], df['Close'][l]) - min(levels, key=lambda x: abs(x - df['Low'][l]))) <= lim
    c3 = max(df['Open'][l], df['Close'][l]) > min(levels, key=lambda x: abs(x - df['Low'][l]))
    c4 = df['High'][l] > min(levels, key=lambda x: abs(x - df['Low'][l]))
    if (c1 or c2) and c3 and c4:
        return min(levels, key=lambda x: abs(x - df['Low'][l]))
    else:
        return 0

def is_below_resistance(l, level_backCandles, level, df): # level_backcandles is our time window
    return df.loc[l-level_backCandles:l-1, 'High'].max() < level # ALL candles  we are considering are below 

def is_above_support(l, level_backCandles, level, df):
    return df.loc[l-level_backCandles:l-1, 'Low'].min() > level

In [ ]:
# ####
# The check_candle_signal function is responsible for combining multiple conditions related to support and resistance levels to determine potential trading signals based on candlestick patterns. Let's break it down in simple terms using the concept of candles:

#     The function takes inputs such as the current candle index l, the number of previous candles to consider for support and resistance detection (n1 and n2), the number of candles to look back for previous levels (levelbackCandles), the number of candles to look back for the window of analysis (windowbackCandles), and the DataFrame (df) containing the candlestick data.

#     Within the function, it initializes TWO EMPTY LISTS, ss and rr, to store support and resistance levels, respectively.

#     It loops through a range of candle indices from l-levelbackCandles to l-n2+1. For each subrow (candle), it checks if it satisfies the conditions for support and resistance using the support and resistance functions. If a subrow meets the conditions for support, its low price is appended to the ss list. If it meets the conditions for resistance, its high price is appended to the rr list.

#     The ss and rr lists are sorted in ascending and descending order, respectively, to keep the lowest support and highest resistance levels when merging close distance levels.

#     The rr and ss lists are then merged into the rrss list to combine support and resistance levels. This is because a support level that is broken can become a resistance level in certain setups.

#     The rrss list is sorted to arrange the levels in ascending order.

#     The closeResistance and closeSupport functions are called to check if the current candle (l) is close enough to any of the levels in the rrss list, using a threshold calculated as df.Close[l]*0.003.

#     Finally, the function evaluates various conditions based on the candlestick patterns and the proximity of the current candle to the detected levels. If a bearish rejection candle is detected (df.rejection[l] == 1) and the current candle is close enough to a resistance level (cR), and all the previous candles within the windowbackCandles range are below the resistance level, it returns 1 as a signal. Similarly, if a bullish rejection candle is detected (df.rejection[l] == 2) and the current candle is close enough to a support level (cS), and all the previous candles within the windowbackCandles range are above the support level, it returns 2 as a signal. Otherwise, it returns 0.

# In summary, the check_candle_signal function combines multiple conditions based on support and resistance levels, as well as candlestick patterns, to identify potential trading signals. It checks if the current candle is close enough to support or resistance levels and if the previous candles are in the anticipated position relative to those levels.


In [152]:
def check_candle_signal(l, n1, n2, levelbackCandles, windowbackCandles, df): # we combine the above conditions, n1 left, n2 right candles, levelbackcandles timewindow before the previous level to detect support or resistance,
    ss = [] #support
    rr = [] #resistance 
    ## the l+1 would be the standard but we need to subtract n2 that are in the future and are not available when we backtest 
    for subrow in range(l-levelbackCandles, l-n2+1): # small gap to avoid look into bias 
        if support(df, subrow, n1, n2):
            ss.append(df.Low[subrow]) 
        if resistance(df, subrow, n1, n2):
            rr.append(df.High[subrow])

#     This code snippet is responsible for merging support levels that are close to each other and keeping only the lowest support level. Here's a simplified explanation:

#     ss.sort() sorts the ss list of support levels in ascending order. This sorting step is necessary because we want to merge support levels that are near each other.

# In summary, this code snippet ensures that the ss list contains only the lowest support levels by merging and removing support levels that are very close to each other. This helps avoid having multiple support levels in close proximity and simplifies the analysis by considering the most relevant support levels.
    ss.sort() #keep lowest support when popping a level,  we do the sort because are going to merge value that are near each other  ex we have t


#     The for loop iterates over the elements in the ss list, starting from index 1.
    for i in range(1,len(ss)):

        # If this condition is true, it means we have reached the end of the list, and the loop is exited using break.
        if(i>=len(ss)):
            break

    #     The if abs(ss[i] - ss[i-1]) / ss[i] <= 0.001 condition checks if the difference between the current support level (ss[i]) and the previous support level (ss[i-1]) is within a certain relative threshold. In this case, the threshold is set to 0.001. This condition ensures that the levels being compared are close to each other.

        if abs(ss[i]-ss[i-1])/ss[i]<=0.001: # merging close distance levels, we avoid too much livelli vicini l un l altro 

 #     If the condition is true, it means that the current support level is very close to the previous support level. In this case, the ss.pop(i) statement removes the current support level from the list, keeping only the smallest support level among the close distance levels.
            ss.pop(i) # we keep the smallest one 

    rr.sort(reverse=True) # keep highest resistance when popping one
    for i in range(1,len(rr)):
        if(i>=len(rr)):
            break
        if abs(rr[i]-rr[i-1])/rr[i]<=0.001: # merging close distance levels
            rr.pop(i)

    #----------------------------------------------------------------------
    # joined levels
    rrss = rr+ss # joined support and resistance because a support level that is broken can become a resistance in some setup
    rrss.sort()
    for i in range(1,len(rrss)):
        if(i>=len(rrss)):
            break
        if abs(rrss[i]-rrss[i-1])/rrss[i]<=0.001: # merging close distance levels
            rrss.pop(i)
    cR = closeResistance(l, rrss, df.Close[l]*0.003, df)
    cS = closeSupport(l, rrss, df.Close[l]*0.003, df)
    #----------------------------------------------------------------------

    # cR = closeResistance(l, rr, 150e-5, df) # originally supp and res where separated by the author
    # cS = closeSupport(l, ss, 150e-5, df)
    #print(rrss,df.Close*0.002)

    # IMPORTANT SUMS UP ALL THE SHEET a bearish rejection candle combined cR a candle that is close enough to a resistance level and all the previous candle are below the resistance level  here I can add all the parameters i need 
    if (df.rejection[l] == 1 and cR and is_below_resistance(l,windowbackCandles,cR, df)):
        return 1
    elif(df.rejection[l] == 2 and cS and is_above_support(l,windowbackCandles,cS, df)):
        return 2
    else:
        return 0

In [153]:
# call the function passing parameters 
from tqdm import tqdm

n1 = 8
n2 = 8
levelbackCandles = 60
windowbackCandles = n2

signal = [0 for i in range(len(data))]

for row in tqdm(range(levelbackCandles+n1, len(data)-n2)):
    signal[row] = check_candle_signal(row, n1, n2, levelbackCandles, windowbackCandles, data)

data["signal"] = signal

100%|██████████| 4975/4975 [00:23<00:00, 210.79it/s]


In [154]:
#data[data["signal"]!=0]

In [176]:
data['pointpos'] = data.apply(lambda row: pointpos(row,"signal"), axis=1)
plot_with_signal(data[750:950])

## 4- Backtesting

In [156]:
data.set_index("Date", inplace=True)

In [157]:
data

,Open,High,Low,Close,Volume,rejection,pointpos,signal
Date,,,,,,,,
2003-12-29,1.9308,1.9817,1.9217,1.9750,75676800.0,0,NaN,0
2003-12-30,1.9542,2.0167,1.9525,1.9792,55118400.0,0,NaN,0
2003-12-31,1.9783,1.9875,1.9017,1.9333,49736400.0,0,NaN,0
2004-01-02,1.9642,1.9908,1.9233,1.9233,43640400.0,0,NaN,0
2004-01-05,1.9525,1.9992,1.9350,1.9858,57544800.0,0,NaN,0
...,...,...,...,...,...,...,...,...
2024-01-22,600.4600,603.3100,590.7000,596.5400,29839304.0,0,NaN,0
2024-01-23,595.7000,599.1000,585.8500,598.7300,20613182.0,2,NaN,0
2024-01-24,603.0400,628.4900,599.3800,613.6200,40854490.0,0,NaN,0


In [177]:
#data[data["signal"]!=0]


,Open,High,Low,Close,Volume,rejection,pointpos,signal,ATR,RSI
Date,,,,,,,,,,
2006-09-13,4.8233,4.9133,4.8083,4.8500,47647800.0,1,4.9134,1,0.205797,70.436331
2008-10-29,1.9925,2.2000,1.9825,2.0700,97753200.0,1,2.2001,1,0.215953,71.639700
2009-12-03,3.4700,3.5325,3.4550,3.4575,16880000.0,1,3.5326,1,0.117933,79.629286
2010-06-25,2.7975,2.7975,2.7250,2.7700,13101600.0,2,2.7249,2,0.141389,28.484550
2012-03-27,3.6875,3.7475,3.6875,3.7050,7844400.0,1,3.7476,1,0.089775,67.824113
2012-06-20,3.3400,3.4250,3.3250,3.3625,20904400.0,1,3.4251,1,0.117532,81.082692
2013-02-13,3.1150,3.1650,3.0850,3.0925,18485200.0,1,3.1651,1,0.067613,49.369757
2013-11-05,3.6775,3.7175,3.6425,3.7000,8766000.0,2,3.6424,2,0.066229,14.879450
2014-05-09,4.4850,4.5275,4.4275,4.5125,14617600.0,2,4.4274,2,0.119795,31.357559


In [159]:
data['ATR'] = pa.atr(high=data.High, low=data.Low, close=data.Close, length=14)
data['RSI'] = pa.rsi(data.Close, length=5)

In [184]:
def SIGNAL():
    return data.signal

In [161]:
#A new strategy needs to extend Strategy class and override its two abstract methods: init() and next().
#Method init() is invoked before the strategy is run. Within it, one ideally precomputes in efficient, 
#vectorized manner whatever indicators and signals the strategy depends on.
#Method next() is then iteratively called by the Backtest instance, once for each data point (data frame row), 
#simulating the incremental availability of each new full candlestick bar.

#Note, backtesting.py cannot make decisions / trades within candlesticks — any new orders are executed on the
#next candle's open (or the current candle's close if trade_on_close=True). 
#If you find yourself wishing to trade within candlesticks (e.g. daytrading), you instead need to begin 
#with more fine-grained (e.g. hourly) data.

### 4.1- Using Fixed SL and TP rules

In [186]:

# Trader fixed SL and TP
from backtesting import Strategy, Backtest

class MyCandlesStrat(Strategy):  
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)
        self.ratio = 2 # take profit ratio is equal to 2
        self.risk_perc = 0.1 #  sl and take profit are dependent by the price itself a 1 percent

    def next(self):
        super().next() 
        if self.signal1==2: #bullish
            #stop loss is equal to closing price minus a small perc
            sl1 = self.data.Close[-1] - self.data.Close[-1]*self.risk_perc
            tp1 = self.data.Close[-1] + (self.data.Close[-1]*self.risk_perc)*self.ratio
            self.buy(sl=sl1, tp=tp1) #  buy
        elif self.signal1==1:
            sl1 = self.data.Close[-1] + self.data.Close[-1]*self.risk_perc
            tp1 = self.data.Close[-1] - (self.data.Close[-1]*self.risk_perc)*self.ratio
            self.sell(sl=sl1, tp=tp1)

data.index = pd.to_datetime(data.index) #this line is essential if use csv files           
bt = Backtest(data, MyCandlesStrat, cash=100_000, commission=.002)
stat = bt.run()
stat

/var/folders/k5/h1ckqhvs6kb6sb1p9b_90n1w0000gn/T/ipykernel_73358/2809907236.py:25: UserWarning:

Data index is not sorted in ascending order. Sorting.



Start                     2003-12-29 00:00:00
End                       2024-01-26 00:00:00
Duration                   7333 days 00:00:00
Exposure Time [%]                    8.631954
Equity Final [$]                 71492.484514
Equity Peak [$]                 102083.095069
Return [%]                         -28.507515
Buy & Hold Return [%]            30801.772152
Return (Ann.) [%]                   -1.660298
Volatility (Ann.) [%]                9.087316
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -40.208991
Avg. Drawdown [%]                  -23.681826
Max. Drawdown Duration     6324 days 00:00:00
Avg. Drawdown Duration     3172 days 00:00:00
# Trades                                   11
Win Rate [%]                        27.272727
Best Trade [%]                      21.426577
Worst Trade [%]                    -12.485779
Avg. Trade [%]                    

In [163]:
bt.plot()

/opt/homebrew/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'

/opt/homebrew/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'

/opt/homebrew/lib/python3.10/site-packages/backtesting/_plotting.py:659: UserWarning:

found multiple competing values for 'toolbar.active_drag' property; using the latest value

/opt/homebrew/lib/python3.10/site-packages/backtesting/_plotting.py:659: UserWarning:

found multiple competing values for 'toolbar.active_scroll' property; using the latest value



GridPlot(id='p5923', ...)

### 4.2- Using the RSI for Exit Signals

In [164]:
from backtesting import Strategy, Backtest

class MyCandlesStrat(Strategy):
    ratio = 1.5
    risk_perc = 0.1  
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)
        #self.ratio
        #self.risk_perc

    def next(self):
        super().next()
        
        if len(self.trades)>0: # if i have open trades 
            if self.trades[-1].is_long and self.data.RSI[-1]>=80:
                self.trades[-1].close()
            elif self.trades[-1].is_short and self.data.RSI[-1]<=20:
                self.trades[-1].close()

        if self.signal1==2 and len(self.trades)==0:
            sl1 = self.data.Close[-1] - self.data.Close[-1]*self.risk_perc
            tp1 = self.data.Close[-1] + (self.data.Close[-1]*self.risk_perc)*self.ratio
            self.buy(sl=sl1, tp=tp1)
        elif self.signal1==1 and len(self.trades)==0:
            sl1 = self.data.Close[-1] + self.data.Close[-1]*self.risk_perc
            tp1 = self.data.Close[-1] - (self.data.Close[-1]*self.risk_perc)*self.ratio
            self.sell(sl=sl1, tp=tp1)
bt = Backtest(data, MyCandlesStrat, cash=100_000, commission=.05)
stat = bt.run()
stat

/var/folders/k5/h1ckqhvs6kb6sb1p9b_90n1w0000gn/T/ipykernel_73358/2568147000.py:29: UserWarning:

Data index is not sorted in ascending order. Sorting.



Start                     2003-12-29 00:00:00
End                       2024-01-26 00:00:00
Duration                   7333 days 00:00:00
Exposure Time [%]                    4.019006
Equity Final [$]                 28250.539545
Equity Peak [$]                      100000.0
Return [%]                          -71.74946
Buy & Hold Return [%]            30801.772152
Return (Ann.) [%]                   -6.111778
Volatility (Ann.) [%]                8.144261
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -71.74946
Avg. Drawdown [%]                   -71.74946
Max. Drawdown Duration     6344 days 00:00:00
Avg. Drawdown Duration     6344 days 00:00:00
# Trades                                   12
Win Rate [%]                             25.0
Best Trade [%]                        7.43158
Worst Trade [%]                    -18.169272
Avg. Trade [%]                    

In [165]:
bt.plot()

/opt/homebrew/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'

/opt/homebrew/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'

/opt/homebrew/lib/python3.10/site-packages/backtesting/_plotting.py:659: UserWarning:

found multiple competing values for 'toolbar.active_drag' property; using the latest value

/opt/homebrew/lib/python3.10/site-packages/backtesting/_plotting.py:659: UserWarning:

found multiple competing values for 'toolbar.active_scroll' property; using the latest value



GridPlot(id='p6304', ...)

In [166]:
# Define a range of values to test for each parameter
param_grid = {'ratio': list(np.arange(1.5, 3.5, 0.5)), 'risk_perc': list(np.arange(0.06, 0.2, 0.02))}
# Run the optimization
res = bt.optimize(**param_grid, random_state=5)

/opt/homebrew/lib/python3.10/site-packages/backtesting/backtesting.py:1375: UserWarning:

For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.



In [167]:
# Print the best results and the parameters that lead to these results
print("Best result: ", res['Return [%]'])
print("Parameters for best result: ", res['_strategy'])

Best result:  -64.20646168000006
Parameters for best result:  MyCandlesStrat(ratio=2.0,risk_perc=0.14)


### 4.3- ATR based SL and TP

In [168]:
# ATR related SL and TP
from backtesting import Strategy, Backtest
import numpy as np

class MyCandlesStrat(Strategy): 
    atr_f = 3
    ratio_f = 2
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next() 
        if self.signal1==2:
            sl1 = self.data.Close[-1] - self.data.ATR[-1]*self.atr_f
            tp1 = self.data.Close[-1] + self.data.ATR[-1]*self.ratio_f*self.atr_f
            self.buy(sl=sl1, tp=tp1)
        elif self.signal1==1:
            sl1 = self.data.Close[-1] + self.data.ATR[-1]*self.atr_f
            tp1 = self.data.Close[-1] - self.data.ATR[-1]*self.ratio_f*self.atr_f
            self.sell(sl=sl1, tp=tp1)
bt = Backtest(data, MyCandlesStrat, cash=100_000, commission=.000)
stat = bt.run()
stat

/var/folders/k5/h1ckqhvs6kb6sb1p9b_90n1w0000gn/T/ipykernel_73358/1112866455.py:22: UserWarning:

Data index is not sorted in ascending order. Sorting.



Start                     2003-12-29 00:00:00
End                       2024-01-26 00:00:00
Duration                   7333 days 00:00:00
Exposure Time [%]                    8.928925
Equity Final [$]                 46406.326634
Equity Peak [$]                 114973.913592
Return [%]                         -53.593673
Buy & Hold Return [%]            30801.772152
Return (Ann.) [%]                   -3.757883
Volatility (Ann.) [%]               14.085945
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -61.340699
Avg. Drawdown [%]                  -28.572124
Max. Drawdown Duration     5545 days 00:00:00
Avg. Drawdown Duration     2113 days 00:00:00
# Trades                                   11
Win Rate [%]                        27.272727
Best Trade [%]                      15.672113
Worst Trade [%]                     -24.24496
Avg. Trade [%]                    

In [169]:
bt.plot()

/opt/homebrew/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'

/opt/homebrew/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'

/opt/homebrew/lib/python3.10/site-packages/backtesting/_plotting.py:659: UserWarning:

found multiple competing values for 'toolbar.active_drag' property; using the latest value

/opt/homebrew/lib/python3.10/site-packages/backtesting/_plotting.py:659: UserWarning:

found multiple competing values for 'toolbar.active_scroll' property; using the latest value



GridPlot(id='p6685', ...)

### 4.4- Trail Stop

In [170]:
#fixed distance Trailing SL
from backtesting import Strategy, Backtest

class MyCandlesStrat(Strategy):
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next()
        sltr=self.data.Close[-1]*0.02

        for trade in self.trades: 
            if trade.is_long: 
                # we chase the price to move the sl higher
                # max between 1) trade.sl( on infinite if not available trade.sl and 2) closing price minus sl
                trade.sl = max(trade.sl or -np.inf, self.data.Close[-1] - sltr)
            else:
                trade.sl = min(trade.sl or np.inf, self.data.Close[-1] + sltr) 
        
        if self.signal1==2 and len(self.trades)==0: 
            sl1 = self.data.Close[-1] - sltr
            self.buy(sl=sl1)
        elif self.signal1==1 and len(self.trades)==0: 
            sl1 = self.data.Close[-1] + sltr
            self.sell(sl=sl1)


bt = Backtest(data, MyCandlesStrat, cash=100_000, commission=.000)
stat = bt.run()
stat

/var/folders/k5/h1ckqhvs6kb6sb1p9b_90n1w0000gn/T/ipykernel_73358/2359968017.py:29: UserWarning:

Data index is not sorted in ascending order. Sorting.



Start                     2003-12-29 00:00:00
End                       2024-01-26 00:00:00
Duration                   7333 days 00:00:00
Exposure Time [%]                    0.415759
Equity Final [$]                 91880.497988
Equity Peak [$]                      100000.0
Return [%]                          -8.119502
Buy & Hold Return [%]            30801.772152
Return (Ann.) [%]                   -0.421594
Volatility (Ann.) [%]                1.925922
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -8.152709
Avg. Drawdown [%]                   -8.152709
Max. Drawdown Duration     6344 days 00:00:00
Avg. Drawdown Duration     6344 days 00:00:00
# Trades                                   12
Win Rate [%]                             25.0
Best Trade [%]                       2.553571
Worst Trade [%]                     -3.421311
Avg. Trade [%]                    

In [171]:
bt.plot()

/opt/homebrew/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'

/opt/homebrew/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'

/opt/homebrew/lib/python3.10/site-packages/backtesting/_plotting.py:659: UserWarning:

found multiple competing values for 'toolbar.active_drag' property; using the latest value

/opt/homebrew/lib/python3.10/site-packages/backtesting/_plotting.py:659: UserWarning:

found multiple competing values for 'toolbar.active_scroll' property; using the latest value



GridPlot(id='p7066', ...)

In [172]:
#ATR based Trailing Stop
from backtesting import Strategy, Backtest

class MyCandlesStrat(Strategy):
    atr_f = 0.6
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)
        self.sltr=0

    def next(self):
        super().next()
        
        for trade in self.trades: 
            if trade.is_long: 
                trade.sl = max(trade.sl or -np.inf, self.data.Close[-1] - self.sltr)
            else:
                trade.sl = min(trade.sl or np.inf, self.data.Close[-1] + self.sltr)

        if self.signal1==2 and len(self.trades)==0: 
            self.sltr=self.data.ATR[-1]/self.atr_f
            sl1 = self.data.Close[-1] - self.data.ATR[-1]/self.atr_f
            self.buy(sl=sl1)
        elif self.signal1==1 and len(self.trades)==0: 
            self.sltr=self.data.ATR[-1]/self.atr_f
            sl1 = self.data.Close[-1] + self.data.ATR[-1]/self.atr_f
            self.sell(sl=sl1)
bt = Backtest(data, MyCandlesStrat, cash=100_000, commission=.000)
stat = bt.run()
stat

/var/folders/k5/h1ckqhvs6kb6sb1p9b_90n1w0000gn/T/ipykernel_73358/3437680106.py:28: UserWarning:

Data index is not sorted in ascending order. Sorting.



Start                     2003-12-29 00:00:00
End                       2024-01-26 00:00:00
Duration                   7333 days 00:00:00
Exposure Time [%]                    1.702633
Equity Final [$]                 66167.842707
Equity Peak [$]                 104773.310551
Return [%]                         -33.832157
Buy & Hold Return [%]            30801.772152
Return (Ann.) [%]                     -2.0393
Volatility (Ann.) [%]                6.397835
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -38.201173
Avg. Drawdown [%]                  -23.924991
Max. Drawdown Duration     5559 days 00:00:00
Avg. Drawdown Duration     3172 days 00:00:00
# Trades                                   12
Win Rate [%]                        16.666667
Best Trade [%]                       6.611315
Worst Trade [%]                    -12.260808
Avg. Trade [%]                    

## 5- Lot sizing and trade management

In [173]:
class MyCandlesStrat(Strategy):
    lotsize = 1 # buying one unity 
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)
        self.ratio = 1.
        self.risk_perc = 0.1

    def next(self):
        super().next() 
        if self.signal1==2 and len(self.trades)==0:
            sl1 = self.data.Close[-1] - self.data.Close[-1]*self.risk_perc
            #first taking profit (lotsize is 1 but we do 2 buy below), this is closer than the second taking profit
            tp1 = self.data.Close[-1] + (self.data.Close[-1]*self.risk_perc)*self.ratio*0.8
            tp2 = self.data.Close[-1] + (self.data.Close[-1]*self.risk_perc)*self.ratio*1.2
            self.buy(sl=sl1, tp=tp1, size=self.lotsize)
            self.buy(sl=sl1, tp=tp2, size=self.lotsize)
        elif self.signal1==1 and len(self.trades)==0:
            sl1 = self.data.Close[-1] + self.data.Close[-1]*self.risk_perc
            tp1 = self.data.Close[-1] - (self.data.Close[-1]*self.risk_perc)*self.ratio*0.8
            tp2 = self.data.Close[-1] - (self.data.Close[-1]*self.risk_perc)*self.ratio*1.2
            self.sell(sl=sl1, tp=tp1, size=self.lotsize)
            self.sell(sl=sl1, tp=tp2, size=self.lotsize)
bt = Backtest(data, MyCandlesStrat, cash=100_000, margin=1/1, commission=.05)
stat = bt.run()
stat

/var/folders/k5/h1ckqhvs6kb6sb1p9b_90n1w0000gn/T/ipykernel_73358/1843021830.py:24: UserWarning:

Data index is not sorted in ascending order. Sorting.



Start                     2003-12-29 00:00:00
End                       2024-01-26 00:00:00
Duration                   7333 days 00:00:00
Exposure Time [%]                    6.830331
Equity Final [$]                  99967.86648
Equity Peak [$]                      100000.0
Return [%]                          -0.032134
Buy & Hold Return [%]            30801.772152
Return (Ann.) [%]                   -0.001603
Volatility (Ann.) [%]                0.002487
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -0.032134
Avg. Drawdown [%]                   -0.032134
Max. Drawdown Duration     6344 days 00:00:00
Avg. Drawdown Duration     6344 days 00:00:00
# Trades                                   22
Win Rate [%]                        31.818182
Best Trade [%]                       8.540973
Worst Trade [%]                    -18.169272
Avg. Trade [%]                    